In [ ]:
import Pkg
Pkg.add("JuMP")
import Pkg; Pkg.add("JSON")
using JSON
import Pkg
Pkg.add("HiGHS")

In [ ]:
# Vs×S
x_values = []

# (E0×Q0)∪(Es×Qs)
y_value = []

# Et
z_values = []

In [11]:
using HiGHS
using JuMP

# Create a JuMP model with HiGHS solver
model = Model(HiGHS.Optimizer)

# Define binary variables 
@variable(model, x[x_values], binary=true)
@variable(model, y[y_values], binary=true)
@variable(model, z[z_values], binary=true)

# Set objective function to maximize
@objective(model, Max, 3x + 2y)

# Add constraints
@constraint(model, constraint1, 2x + y <= 10)
@constraint(model, constraint2, x + 3y <= 12)

# Solve the MILP problem
optimize!(model)

# Check if the optimization was successful
if termination_status(model) == MOI.OPTIMAL
    # Access solution information
    obj_value = objective_value(model)
    x_value = value(x)
    y_value = value(y)

    println("Objective Value: $obj_value")
    println("Optimal x Value: $x_value")
    println("Optimal y Value: $y_value")
else
    println("No optimal solution found.")
end


Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 2 cols, 4 nonzeros
2 rows, 2 cols, 4 nonzeros
Objective function is integral with scale 1

Solving MIP model with:
   2 rows
   2 cols (0 binary, 2 integer, 0 implied int., 0 continuous)
   4 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   23              -inf                 inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   23              13                76.92%        0      0      0         0     0.0s
 R       0       0         0   0.00%   16.4            16                 2.50%        0      0      0         2     0.0s

Solving report
  Status            Optimal
  Primal bound      16
  Dual bound        16
  